In [2]:
!pip install opendatasets --quiet
import opendatasets as od
od.download('https://www.kaggle.com/datasets/andrewmvd/animal-faces')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: shahbozsharafzoda
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/animal-faces


100%|██████████| 696M/696M [00:04<00:00, 167MB/s]


In [5]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import os


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device available",device)


Device available cuda


In [7]:
# By default we for each dataset, we have train and validation. Our task will be to go though all and split into 3: train, validation, test

In [12]:
image_path = []
labels = []

for i in os.listdir("/content/animal-faces/afhq/"):
  for label in os.listdir(f"/content/animal-faces/afhq/{i}"):
    for image in os.listdir(f"/content/animal-faces/afhq/{i}/{label}"):
      labels.append(label) # Append/Put this label in the list
      image_path.append(f"/content/animal-faces/afhq/{i}/{label}/{image}")

data_df = pd.DataFrame(zip(image_path, labels), columns = ['image_paths', 'labels'])
data_df.head() # Visualizing the first 5 rows

,image_paths,labels
0,/content/animal-faces/afhq/train/wild/flickr_w...,wild
1,/content/animal-faces/afhq/train/wild/flickr_w...,wild
2,/content/animal-faces/afhq/train/wild/flickr_w...,wild
3,/content/animal-faces/afhq/train/wild/flickr_w...,wild
4,/content/animal-faces/afhq/train/wild/flickr_w...,wild


In [13]:
print(data_df['labels'].unique())

['wild' 'cat' 'dog']


In [14]:
train = data_df.sample(frac=0.7) # this means for training purpose, we want to use 70% of the whole data
test = data_df.drop(train.index) # 30 % of the data will go to test

val = test.sample(frac=0.5) # on the other hand half of that 30 % will go to validation
test = test.drop(val.index)

print(train.shape)
print(val.shape)
print(test.shape)

(11291, 2)
(2420, 2)
(2419, 2)


In [16]:
label_encoder = LabelEncoder()
label_encoder.fit(data_df['labels']) # here we are asking label encoder to give the values for the labels we have in the dataset(cat,dog,wild)

# cat=0, dog=1, wild=2, it could be in any order

# transform will make all the images to have the same properties: size, data type
transform = transforms.Compose(
    [
        transforms.Resize((128,128)),
        transforms.ToTensor(),
        transforms.ConvertImageDtype(torch.float)

    ]
)